In [5]:
import csv
import json
import time
import urllib
import requests
import pandas as pd
import nltk
import spacy
import string
import numpy as np
from google.colab import drive
drive.mount("/content/drive")

In [39]:
df = pd.read_csv("/content/drive/MyDrive/movie_reviews_preprocessed.csv")
df.tail()

,Unnamed: 0,Unnamed: 0.1,review,rating
1683263,1683263,1683263,after the critical mutilation chappie received...,-1.0
1683264,1683264,1683264,hes a happy chappie so says the character yola...,0.0
1683265,1683265,1683265,i know now a days movies are all about renewin...,0.0
1683266,1683266,1683266,my least favorite from blomkamp so far so chap...,-1.0
1683267,1683267,1683267,i felt emotionally involved during almost all ...,-1.0


In [4]:
#  Drop Nan values
df.dropna(subset = ["rating"], inplace=True)
df.dropna(subset = ["review"], inplace=True)

In [5]:
df = df.reset_index(drop=True)

In [6]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1,  inplace = True)

In [7]:
df.tail()

,review,rating
1683249,after the critical mutilation chappie received...,-1.0
1683250,hes a happy chappie so says the character yola...,0.0
1683251,i know now a days movies are all about renewin...,0.0
1683252,my least favorite from blomkamp so far so chap...,-1.0
1683253,i felt emotionally involved during almost all ...,-1.0


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, SpatialDropout1D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.model_selection import train_test_split


In [9]:
texts = df["review"]
target = df["rating"]

In [10]:
tokenizer = Tokenizer(num_words=10000, oov_token=None)
tokenizer.fit_on_texts(texts)

X = tokenizer.texts_to_sequences(texts)
X, y = np.array(X), np.array(target)

X = pad_sequences(X, maxlen=100)
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size=0.20, random_state=1)

In [11]:
lrr = ReduceLROnPlateau(monitor='val_accuracy', factor=.01,  patience=3, min_lr=1e-5)

In [12]:
vocab_size = 10000
embedding_dim = 64

In [13]:
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(Y_train, 3)
Y_val = to_categorical(Y_val, 3)

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, dropout=0.2)),
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 195       
Total params: 714,499
Trainable params: 714,499
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 10

In [16]:
hist = model.fit(X_train, Y_train, batch_size=32, epochs=num_epochs, verbose=1, validation_data=(X_val, Y_val))

Epoch 1/10
42082/42082 [==============================] - 808s 19ms/step - loss: 0.5843 - accuracy: 0.7578 - val_loss: 0.5425 - val_accuracy: 0.7779
Epoch 2/10
42082/42082 [==============================] - 807s 19ms/step - loss: 0.5268 - accuracy: 0.7837 - val_loss: 0.5261 - val_accuracy: 0.7839
Epoch 3/10
42082/42082 [==============================] - 798s 19ms/step - loss: 0.5067 - accuracy: 0.7931 - val_loss: 0.5211 - val_accuracy: 0.7860
Epoch 4/10
42082/42082 [==============================] - 799s 19ms/step - loss: 0.4938 - accuracy: 0.7987 - val_loss: 0.5230 - val_accuracy: 0.7875
Epoch 5/10
42082/42082 [==============================] - 804s 19ms/step - loss: 0.4847 - accuracy: 0.8026 - val_loss: 0.5312 - val_accuracy: 0.7854
Epoch 6/10
42082/42082 [==============================] - 806s 19ms/step - loss: 0.4782 - accuracy: 0.8055 - val_loss: 0.5233 - val_accuracy: 0.7863
Epoch 7/10
42082/42082 [==============================] - 799s 19ms/step - loss: 0.4732 - accuracy: 0.8080

In [17]:
model.save('/content/drive/MyDrive/movie_review_model.h5')

In [38]:
tokenizer = Tokenizer(num_words=100, oov_token=None)
d = pd.DataFrame({"review": ["Worst thing I have ever seen", "Worst"], "ratin"})
tokenizer.fit_on_texts(d['review'])
e = tokenizer.texts_to_sequences(d)
e = pad_sequences(e, maxlen=100)
e = np.array(e)
e

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [30]:
model.predict(e)

array([[0.15888655, 0.5672802 , 0.27383322]], dtype=float32)

In [65]:
dff = df.copy()[: 20]

In [66]:
dff.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1,  inplace = True)

In [67]:
from tensorflow.keras.utils import to_categorical


tokenizer = Tokenizer(num_words=10000, oov_token=None)
tokenizer.fit_on_texts(dff['review'])

X = tokenizer.texts_to_sequences(dff['review'])
X, y = np.array(X), np.array(dff['rating'])

y = to_categorical(y, num_classes = 3)

X = pad_sequences(X, maxlen=100)
y

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [69]:
model.evaluate(X, y)

In [59]:
dff

,review,rating
0,altered follows the capture of an extraterrest...,1.0
1,although it would be a stretch to call it orig...,0.0
2,going into altered i was aware of a couple thi...,1.0
3,like scifi like gore if like me the answer is ...,1.0
4,i watched it expecting it to be a cheap low bu...,1.0
5,potential spoilers i thought it started off ve...,0.0
6,after being taken by aliens 15 years ago a gro...,1.0
7,highconcept with tight wellcontrolled shots al...,1.0
8,after reading the first 5 reviews on imdb i wa...,-1.0
9,cody duke otispaul mccarthyboyington brad wil...,0.0
